In [ ]:
# One must patch the DPO Trainer first!
from unsloth import PatchDPOTrainer

PatchDPOTrainer()

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/gz-fs/models/Qwen3-8B", # Choose ANY! eg mistralai/Mistral-7B-Instruct-v0.2
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
# Load and split dataset 

import json
from datasets import Dataset
from sklearn.model_selection import train_test_split

# Load original data
dpo_dataset_path = "/root/tiktok_techjam/policy_learning/dataset/dpo_dataset_hf.json"
with open(dpo_dataset_path, 'r', encoding='utf-8') as f:
    raw_data = json.load(f)

# Process data format
processed_data = []
for item in raw_data:
    processed_data.append({
        "prompt": item["prompt"],
        "chosen": item["chosen"],
        "rejected": item["rejected"]
    })

print(f"Total dataset size: {len(processed_data)}")

# Dataset split (80% tra    in, 20% eval)
train_data, eval_data = train_test_split(
    processed_data, 
    test_size=0.2, 
    random_state=42,
    shuffle=True
)

print(f"Train dataset size: {len(train_data)}")
print(f"Eval dataset size: {len(eval_data)}")

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_list(train_data)
eval_dataset = Dataset.from_list(eval_data)

# Create raw_datasets dictionary
raw_datasets = {
    "train": train_dataset,
    "test": eval_dataset
}

print("Dataset split and loading completed!")
print(f"Train dataset columns: {train_dataset.column_names}")
print(f"Train dataset sample:")
print(train_dataset[0])


In [ ]:
# Check dataset status
print("=== Dataset Information ===")
print(f"Training dataset: {train_dataset}")
print(f"Evaluation dataset: {eval_dataset}")
print("\n=== Sample Data ===")
print("Training sample:")
for key, value in train_dataset[0].items():
    print(f"{key}: {value[:100]}..." if len(str(value)) > 100 else f"{key}: {value}")
    
print(f"\nDataset ready for DPO training!")

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 64,
    lora_dropout = 0, # Currently only supports dropout = 0
    bias = "none",    # Currently only supports bias = "none"
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
from transformers import TrainingArguments
from trl import DPOTrainer, DPOConfig

# 初始化DPO训练器
dpo_trainer = DPOTrainer(
    model = model,
    ref_model = None,
    args = DPOConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.1,
        num_train_epochs = 10,
        learning_rate = 5e-6,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.0,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
    beta = 0.1,
    train_dataset = raw_datasets["train"],
    eval_dataset = raw_datasets["test"],
    tokenizer = tokenizer,
    max_length = 1024,
    max_prompt_length = 512,
)

print("DPO Trainer initialized successfully!")
print(f"Training samples: {len(raw_datasets['train'])}")
print(f"Evaluation samples: {len(raw_datasets['test'])}")

In [ ]:
# Start DPO training
print("Starting DPO training...")
print(f"Total training steps: {len(train_dataset) // (2 * 4) * 3}")  # batch_size * grad_accum * epochs

# Train model
trainer_output = dpo_trainer.train()

print("Training completed!")
print(f"Final training loss: {trainer_output.training_loss}")

# Save final model
print("Saving final model...")
dpo_trainer.save_model("./final_dpo_model")
tokenizer.save_pretrained("./final_dpo_model")
print("Model saved successfully!")

In [7]:
# load the train model

model_name = "/root/tiktok_techjam/policy_learning/final_dpo_model"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=4096,
    dtype="bfloat16",
    load_in_4bit=True,
)

==((====))==  Unsloth 2025.8.10: Fast Qwen3 patching. Transformers: 4.55.4.
   \\   /|    NVIDIA A100-PCIE-40GB. Num GPUs = 1. Max memory: 39.495 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
# inference on the test dataset
from unsloth import FastLanguageModel
from transformers import TextStreamer

inputs = tokenizer(
    ["作为TikTok收益分配专家，请为以下创作者推荐一个公平的收益分成比例。\n\n创作者档案：\n- 创作经验：1个月\n- 粉丝数量：50000\n- 历史表现：0.57\n- 稳定性评分：0.37\n\n内容质量评估：\n- 内容价值：0.83\n- 视觉质量：0.59\n- 主播表现：0.66\n- 整体评分：0.69\n\n互动表现：\n- 观看人数：14574\n- 参与率：0.12\n- 留存率：0.58\n- 聊天活跃度：0.44\n\n基础数据：\n- 直播时长：120分钟\n- 峰值观众：14574\n- 平均观众：12515\n- 评论数：23523\n- 点赞数：29275\n\n提供一个公平的创作者收益分成比例(0-100%)和简短的理由"],
    return_tensors="pt",
).to(model.device)

FastLanguageModel.for_inference(model) 
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 1024)

作为TikTok收益分配专家，请为以下创作者推荐一个公平的收益分成比例。

创作者档案：
- 创作经验：1个月
- 粉丝数量：50000
- 历史表现：0.57
- 稳定性评分：0.37

内容质量评估：
- 内容价值：0.83
- 视觉质量：0.59
- 主播表现：0.66
- 整体评分：0.69

互动表现：
- 观看人数：14574
- 参与率：0.12
- 留存率：0.58
- 聊天活跃度：0.44

基础数据：
- 直播时长：120分钟
- 峰值观众：14574
- 平均观众：12515
- 评论数：23523
- 点赞数：29275

提供一个公平的创作者收益分成比例(0-100%)和简短的理由，以及建议的改进

方向。
作为TikTok收益分配专家，请为以下创作者推荐一个公平的收益分成比例。

创作者档案：
- 创作经验：1个月
- 絲粉數量：50000
- 歷史表現：0.57
- 穩定性評分：0.37

內容質量評估：
- 內容價值：0.83
- 美術質量：0.59
- 主播表現：0.66
- 整體評分：0.69

互動表現：
- 觀看人數：14574
- 參與率：0.12
- 留存率：0.58
- 聊天活躍度：0.44

基礎數據：
- 直播時長：120分鐘
- 峰值觀眾：14574
- 平均觀眾：12515
- 評論數：23523
- 点赞数：29275

提供一个公平的创作者收益分成比例(0-100%)和简短的理由，以及建议的改进方向。

好的，我现在需要帮用户解决一个关于TikTok创作者收益分成比例的问题。首先，我得仔细看看用户提供的所有数据，然后根据这些数据来推荐一个公平的比例，同时还要给出理由和改进建议。

首先，用户提供的数据包括创作者的经验、粉丝数量、历史表现、稳定性评分，还有内容质量评估、互动表现和基础数据。我需要把这些数据综合起来分析。

创作者有1个月的经验，粉丝数5万，这说明他可能是一个新晋的创作者，虽然粉丝不算太少，但经验不足。历史表现是0.57，稳定性评分0.37，这两个分数可能指的是某些评分系统中的数值，比如TikTok的内部评分，但具体标准我不太清楚，不过通常0.57可能属于中等水平，而稳定性评分0.37可能较低，说明直播的稳定性不够好，可能有中断或者不规律的情况。

内容质量方面，内容价值0.83，这还不错，说明内容有较高的价值；美术质量0.59，可能在视觉效果上还有提升空间；主播表现0.66，整体评分0
